### ChatGPT Use Case Addition

pip install openai

In [3]:
import os
import re
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from openai import OpenAI
import time  

#client = OpenAI(api_key= 'secret key')


folder_path = "raw_data/announcements"
html_files = [f for f in os.listdir(folder_path) if f.endswith(".html")]

def extract_date_from_filename(filename):
    match = re.search(r"monetary(\d{8})", filename)
    if match:
        return datetime.strptime(match.group(1), "%Y%m%d")
    return None

dated_files = [(extract_date_from_filename(f), f) for f in html_files]
dated_files = [pair for pair in dated_files if pair[0] is not None]
dated_files.sort()  

ratings = []

for date, filename in dated_files:
    try:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            fomc_text = soup.get_text(separator=' ', strip=True)
        
        prompt = f"""
        You are a financial analyst specializing in monetary policy communications. Read the following FOMC announcement and answer the following questions:

        1. What is the sentiment at the beginning of the announcement? (Bearish, Neutral, Bullish)
        2. Provide a final numerical sentiment rating to the entire document based on your analysis of the tone shift and sentiment throughout, using this scale:
            -1 = Bearish
             0 = Neutral
             1 = Bullish
             The rating can be a decimal (e.g., -0.9,-0.8,-.0.7,-0.6,-0.5, -0.4,-0.3,-0.2,-0.1, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7,0.8, 0.9, etc.). 

        Just output the rating number without any explanation.

        FOMC Text:
        {fomc_text}
        """

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.4
        )

        response_text = response.choices[0].message.content
        match = re.search(r"2\.\s*(-?\d*\.\d+|\d+)", response_text)

        if match:
            extracted_rating = float(match.group(1))
        else:
            extracted_rating = None  

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        extracted_rating = None

    ratings.append(extracted_rating)
    time.sleep(1.2)  

statements_chat_df = pd.DataFrame(ratings, columns=["Sentiment_Rating"])
print(statements_chat_df)
print(f"\nTotal processed: {len(statements_chat_df)}")


     Sentiment_Rating
0                -0.7
1                -0.7
2                -0.7
3                -0.2
4                -0.2
..                ...
195               0.1
196              -0.1
197              -0.1
198               0.2
199               0.1

[200 rows x 1 columns]

Total processed: 200


In [22]:
statements_chat_df_filled = statements_chat_df.copy()
statements_chat_df_filled = statements_chat_df_filled.fillna(0)
statements_chat_df_filled.insert(0, 'meeting_id', range(1, len(statements_chat_df_filled) + 1))
statements_chat_df_filled['document_type'] = 'statement'

In [23]:
statements_chat_df_filled.to_csv('raw_data/statements_chat_df.csv', index = False)

In [11]:
import os
import re
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from openai import OpenAI
import time  

#client = OpenAI(api_key= 'secret key')

folder_path = "raw_data/intermeeting"
html_files = [f for f in os.listdir(folder_path) if f.endswith(".html")]

def extract_date_from_filename(filename):
    match = re.search(r"fomcminutes(\d{8})", filename)
    if match:
        return datetime.strptime(match.group(1), "%Y%m%d")
    return None

dated_files = [(extract_date_from_filename(f), f) for f in html_files]
dated_files = [pair for pair in dated_files if pair[0] is not None]
dated_files.sort()  

ratings = []

for date, filename in dated_files:
    try:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            fomc_text = soup.get_text(separator=' ', strip=True)
        
        if len(fomc_text) > 32000:
            fomc_text = fomc_text[:32000]
            
        prompt = f"""
        You are a financial analyst specializing in monetary policy communications. Read the following FOMC announcement and answer the following questions:

        1. What is the sentiment at the beginning of the announcement? (Bearish, Neutral, Bullish)
        2. Provide a final numerical sentiment rating to the entire document based on your analysis of the tone shift and sentiment throughout, using this scale:
            -1 = Bearish
             0 = Neutral
             1 = Bullish
             The rating can be a decimal (e.g., -0.9,-0.8,-.0.7,-0.6,-0.5, -0.4,-0.3,-0.2,-0.1, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7,0.8, 0.9, etc.). 

        Just output the rating number without any explanation.

        FOMC Text:
        {fomc_text}
        """

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.4
        )

        response_text = response.choices[0].message.content
        match = re.search(r"2\.\s*(-?\d*\.\d+|\d+)", response_text)

        if match:
            extracted_rating = float(match.group(1))
        else:
            extracted_rating = None  

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        extracted_rating = None

    ratings.append(extracted_rating)
    time.sleep(1.2)  

intermeeting_chat_df = pd.DataFrame(ratings, columns=["Sentiment_Rating"])
print(intermeeting_chat_df)
print(f"\nTotal processed: {len(intermeeting_chat_df)}")


Error processing fomcminutes20071031.html: 'utf-8' codec can't decode byte 0x9a in position 7588: invalid start byte
Error processing fomcminutes20080130.html: 'utf-8' codec can't decode byte 0x93 in position 61922: invalid start byte
Error processing fomcminutes20091216.html: 'utf-8' codec can't decode byte 0x92 in position 43345: invalid start byte
Error processing fomcminutes20100127.html: 'utf-8' codec can't decode byte 0x97 in position 51191: invalid start byte
Error processing fomcminutes20100428.html: 'utf-8' codec can't decode byte 0x9a in position 8700: invalid start byte
Error processing fomcminutes20100623.html: 'utf-8' codec can't decode byte 0x96 in position 60390: invalid start byte
Error processing fomcminutes20110126.html: 'utf-8' codec can't decode byte 0x96 in position 35385: invalid start byte
Error processing fomcminutes20110427.html: 'utf-8' codec can't decode byte 0x96 in position 21699: invalid start byte
Error processing fomcminutes20110809.html: 'utf-8' codec c

In [24]:
intermeeting_chat_df_filled = intermeeting_chat_df.copy()
intermeeting_chat_df_filled = intermeeting_chat_df_filled.fillna(0)
intermeeting_chat_df_filled.insert(0, 'meeting_id', range(1, len(intermeeting_chat_df_filled) + 1))
intermeeting_chat_df_filled['document_type'] = 'intermeeting'

In [25]:
intermeeting_chat_df_filled.to_csv('raw_data/intermeeting_chat_df.csv', index = False)